In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**stores.csv**
This file contains anonymized information about the 45 stores, indicating the type and size of store.

**train.csv**
This is the historical training data, which covers to 2010-02-05 to 2012-11-01. Within this file you will find the following fields:

**Store** - the store number <br>
**Dept** - the department number <br>
**Date** - the week <br>
**Weekly_Sales** -  sales for the given department in the given store <br>
**IsHoliday** - whether the week is a special holiday week <br>

**test.csv**
This file is identical to train.csv, except we have withheld the weekly sales. You must predict the sales for each triplet of store, department, and date in this file.

**features.csv**
This file contains additional data related to the store, department, and regional activity for the given dates. It contains the following fields:

**Store** - the store number <br>
**Date** - the week <br>
**Temperature** - average temperature in the region <br>
**Fuel_Price** - cost of fuel in the region <br>
**MarkDown1-5** - anonymized data related to promotional markdowns that Walmart is running. MarkDown data is only available after Nov 2011, and is not available for all stores all the time. Any missing value is marked with an NA. <br>
**CPI** - the consumer price index <br>
**Unemployment** - the unemployment rate <br>
**IsHoliday** - whether the week is a special holiday week <br>

For convenience, the four holidays fall within the following weeks in the dataset (not all holidays are in the data): <br>
*Super Bowl:* 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13  <br>
*Labor Day:* 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13  <br>
*Thanksgiving:* 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13 <br>
*Christmas:* 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13 <br>

# Carregando os dados

In [ ]:
df_train=pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/train.csv.zip')
df_test=pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/test.csv.zip')
df_feat=pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/features.csv.zip')
df_sample_sub=pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip')
df_stores=pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/stores.csv')

In [ ]:
df_sample_sub

In [ ]:
df_train.head(5)

In [ ]:
df_feat.head(5)

In [ ]:
df_stores.head(5)

In [ ]:
# Juntando todos os dfs com o df de treino
df_data_train = df_train.merge(df_feat, on=['Store', 'Date'], how='left').merge(df_stores, on=['Store'], how='left')
df_data_train.shape

In [ ]:
df_data_train

In [ ]:
# Juntando todos os dfs com o df de teste
df_data_test = df_test.merge(df_feat, on=['Store', 'Date'], how='left').merge(df_stores, on=['Store'], how='left')
df_data_test.shape

In [ ]:
df_data_train.info()

In [ ]:
df_data_test.info()

# EDA

In [ ]:
# Convertendo a coluna Data em tipo data.
df_data_train['Date'] = pd.to_datetime(df_data_train['Date'])
df_data_test['Date'] = pd.to_datetime(df_data_test['Date'])

In [ ]:
# Adicionando algumas variáveis de Data
df_data_train['Year'] = df_data_train['Date'].dt.year
df_data_train['Month'] = df_data_train['Date'].dt.month
df_data_train['Week'] = df_data_train['Date'].dt.week
df_data_train['Day'] = df_data_train['Date'].dt.day

df_data_test['Year'] = df_data_test['Date'].dt.year
df_data_test['Month'] = df_data_test['Date'].dt.month
df_data_test['Week'] = df_data_test['Date'].dt.week
df_data_test['Day'] = df_data_test['Date'].dt.day

In [ ]:
df_data_train.info()

In [ ]:
df_data_test.info()

In [ ]:
# Analisando as informações do dataframe
df_data_train.describe()

In [ ]:
# Analisando as informações do dataframe
df_data_test.describe()

Pela descrição das informações do dataframe acima, podemos perceber que a variável **'Weekly_Sales'** tem um valor mínimo negativo. Abaixo vamos analisar um pouco mais a fundo essa variável bem como as outras.

In [ ]:
df_data_train.Weekly_Sales.hist(bins=100)

A variável **Weekly_Sales** não possui distribuição normal.

In [ ]:
df_data_train.Weekly_Sales.groupby(df_data_train.Date.dt.year).plot(figsize=(15,6), legend=True)

Pelo gráfico acima, podemos acompanhar as vendas semanais durante o perído de 2010 à 2012.

In [ ]:
df_data_train.Weekly_Sales.groupby(df_data_train.Date.dt.year==2012).plot(figsize=(15,6), legend=True)

 Podemos notar que no ano de 2012 (na cor laranja), as vendas semanais tiveram um comportamento mais estável em relação aos anos anteriores. E também apresentaram valores mais baixos.

In [ ]:
df_data_train.groupby('Type').count()['Store']

In [ ]:
stores_financ_loss = df_data_train[df_data_train['Weekly_Sales'] < 0]
stores_financ_loss.reset_index(drop=True, inplace=True)
stores_financ_loss

Consideraremos uma perda significativa acima de **$500.00**.

In [ ]:
# Considerando apenas as lojas com perda financeira acima de 500 dólares.
stores_financ_loss[stores_financ_loss.Weekly_Sales <= -500].reset_index(drop=True)

In [ ]:
loss_500 = stores_financ_loss[stores_financ_loss.Weekly_Sales <= -500].reset_index(drop=True)
loss_500

In [ ]:
loss_500.Store.value_counts()

In [ ]:
loss_500.Dept.value_counts()

In [ ]:
loss_500[loss_500.Store == 20]

Analisando a variável **'Weekly_Sales'**, observamos que há alguns registros com valores negativos. A partir disso, poderíamos (após validarmos com a área de negócios) inferir que esses valores negativos poderiam ser lojas que estão tendo perdas financeiras.
As top 5 lojas com maior perda financeira são: **20**, **10**, **16**, **2** e **28**.
Podemos observar também que a maior parte dessas perdas ocorrem no departamento **47**. Com isso, poderia ser verificado com essas lojas o porquê desse departamento (e também dos outros) apresentar essas perdas significativas. Poderia também ser proposto algumas ações promocionais para promover o departamento, redução do estoque desse departamento ou também o aumento da variedade de produtos do estoque desse departamento.

Tomando como exemplo para nossa análise a loja com a maior quantidade de Vendas Semanais com prejuízo (loja **20**), no período de 2010 e 2011, observamos que o **CPI** teve um pequeno aumento, embora a taxa de desemprego tenha diminuído no mesmo período. E ainda, mesmo com os MarkDowns tendo valores positivos, houve um aumento do prejuízo dessa loja para o departamento **47**. Ou seja, mesmo os clientes tendo um poder de compra menor, mas com uma quantidade de clientes maior empregada, e ainda com as promoções do Walmart, o departamento continua tendo prejuízo. Logo, como dito anteriormente, poderia ser feito um estudo em conjunto com as lojas para que haja alguma(s) ação(ões) para tentar amenizar o efeito de perda nas mesmas.

# Data prep

Verificando quais as variáveis possuem missings, e também retirando os registros em que a variável **Weekly_Sales** possui valores negativos.

In [ ]:
print(df_data_train.columns)

In [ ]:
df_data_train.boxplot(column=['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'])

In [ ]:
df_data_test.boxplot(column=['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'])

In [ ]:
df_data_train.shape

In [ ]:
df_data_test.shape

In [ ]:
# Filtrando os valores das variáveis de MarkDown para que não tenha outliers na base.
df_data_train = df_data_train[((df_data_train['MarkDown1'] >= 0) & (df_data_train['MarkDown1'] <= 40000)) &
                              ((df_data_train['MarkDown2'] >= 0) & (df_data_train['MarkDown2'] <= 40000)) &
                              ((df_data_train['MarkDown3'] >= 0) & (df_data_train['MarkDown3'] <= 40000)) &
                              ((df_data_train['MarkDown4'] >= 0) & (df_data_train['MarkDown4'] <= 40000)) &
                              ((df_data_train['MarkDown5'] >= 0) & (df_data_train['MarkDown5'] <= 40000))]
   

In [ ]:
df_data_test = df_data_test[((df_data_test['MarkDown1'] >= 0) & (df_data_test['MarkDown1'] <= 40000)) &
                            ((df_data_test['MarkDown2'] >= 0) & (df_data_test['MarkDown2'] <= 40000)) &
                            ((df_data_test['MarkDown3'] >= 0) & (df_data_test['MarkDown3'] <= 40000)) &
                            ((df_data_test['MarkDown4'] >= 0) & (df_data_test['MarkDown4'] <= 40000)) &
                            ((df_data_test['MarkDown5'] >= 0) & (df_data_test['MarkDown5'] <= 40000))]
   

In [ ]:
df_data_train.boxplot(column=['Weekly_Sales'])

In [ ]:
#Filtrando apenas os registros com vendas mensais maiores ou iguais a 0.
df_data_train = df_data_train[(df_data_train['Weekly_Sales'] >= 0)]    

In [ ]:
df_data_train.boxplot(column=['Temperature', 'CPI'])
    

In [ ]:
df_data_train.boxplot(column=['Fuel_Price', 'Unemployment'])

In [ ]:
df_data_test.boxplot(column=['Fuel_Price', 'Unemployment'])

In [ ]:
df_data_train = df_data_train[(df_data_train['Unemployment'] >= 0) & (df_data_train['Unemployment'] <= 10)]

In [ ]:
df_data_test = df_data_test[(df_data_test['Unemployment'] >= 0) & (df_data_test['Unemployment'] <= 10)]

In [ ]:
df_data_train.reset_index(drop=True, inplace=True)

In [ ]:
df_data_test.reset_index(drop=True, inplace=True)

In [ ]:
def verify_nans(df):
    return df.isna().sum()

In [ ]:
def fill_nans(df):
    df.MarkDown1.fillna(0, inplace=True)
    df.MarkDown2.fillna(0, inplace=True)
    df.MarkDown3.fillna(0, inplace=True)
    df.MarkDown4.fillna(0, inplace=True)
    df.MarkDown5.fillna(0, inplace=True)
    df.CPI.fillna(0, inplace=True)
    df.Unemployment.fillna(0, inplace=True)
    return df

In [ ]:
verify_nans(df_data_train)
df_data_train = fill_nans(df_data_train)

In [ ]:
verify_nans(df_data_test)
df_data_test = fill_nans(df_data_test)

In [ ]:
df_data_train

Como conceitualmente os MarkDowns são descontos, optamos por preencher os valores NAN com 0, indicando que ali não haveria descontos.

In [ ]:
df_data_train.drop(columns=['IsHoliday_y'], inplace=True)

In [ ]:
df_data_test.drop(columns=['IsHoliday_y'], inplace=True)

In [ ]:
df_data_train.rename(columns={'IsHoliday_x':'IsHoliday'}, inplace=True)

In [ ]:
df_data_test.rename(columns={'IsHoliday_x':'IsHoliday'}, inplace=True)

In [ ]:
df_data_train

In [ ]:
df_data_train_cp = df_data_train.copy()

In [ ]:
df_data_test

In [ ]:
df_corr = pd.DataFrame(data=df_data_train, columns=['Weekly_Sales', 'IsHoliday', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Size'])
plt.figure(figsize=(15, 10))
sns.heatmap(df_corr.corr(), annot=True)
plt.show()

ρ = 0,9 a 1 (positivo ou negativo): correlação muito forte; <br>
ρ = 0,7 a 09 (positivo ou negativo): correlação forte; <br>
ρ = 0,5 a 0,7 (positivo ou negativo): correlação moderada; <br>
ρ = 0,3 a 0,5 (positivo ou negativo): correlação fraca; <br>
ρ = 0 a 0,3 (positivo ou negativo): não possui correlação. <br>

Observando o heatmap com as correlações, podemos perceber que apenas as variáveis **MarkDown1 e MarkDown4 possuem correlação forte.** Logo, ao treinar o modelo é interessante excluírmos uma dessas variáveis. Abaixo optamos por retirar a variável **MarkDown1**.

In [ ]:
df_data_train.drop(columns=['MarkDown1', 'Date'], inplace=True)
df_data_test.drop(columns=['MarkDown1', 'Date'], inplace=True)

Para a modelagem, precisamos também converter as variáveis categóricas em numéricas.

In [ ]:
df_data_train.columns

In [ ]:
cols = ['IsHoliday', 'Type']

df_data_train[cols] = df_data_train[cols].apply(LabelEncoder().fit_transform)
df_data_test[cols] = df_data_test[cols].apply(LabelEncoder().fit_transform)

In [ ]:
df_data_train

In [ ]:
X_train = df_data_train[df_data_train.Year != 2011]
y_train = X_train['Weekly_Sales']
X_test = df_data_train[df_data_train.Year == 2011]
y_test = X_test['Weekly_Sales']

Optamos por utilizar o algoritmo RandomForestRegressor que é um modelo emsemble (é a combinação de mais de um modelo), o que contribuiria para uma boa performance.

In [ ]:
def WMAE(df, real, predicted):
    weights = df.IsHoliday.apply(lambda weight: 5 if weight else 1)
    return np.round(np.sum(weights*abs(real-predicted))/(np.sum(weights)), 2)

In [ ]:
rfr = RandomForestRegressor(random_state=777)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
y_score=rfr.score(X_test, y_test)

print("R2: ",y_score)
rfr_adj_r2 = 1 - (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)*(1-y_score)
print("Adjusted R2: ",rfr_adj_r2)

In [ ]:
WMAE(X_test, y_test, y_pred)

Analisando o erro acima, podemos perceber que o modelo está overfitado. Caso houvesse mais tempo para o desenvolvimento do modelo, uma outra abordagem seria testar outros algoritmos e/ou também alterar as proporções das divisões de x e y.

In [ ]:
X_test.shape

In [ ]:
df_y_pred = pd.DataFrame(data=y_pred, columns=['pred'])
df_y_pred.shape

In [ ]:
df_y_pred

In [ ]:
df_y_pred.plot(figsize=(15,8), legend=True)

# Formato submissão

In [ ]:
result = pd.DataFrame()
result['Store'] = X_test['Store'].copy()
result['Dept'] = X_test['Dept'].copy()
result['Year'] = X_test['Year'].copy()
result['Month'] = X_test['Month'].copy()
result['Day'] = X_test['Day'].copy()
result['pred'] = df_y_pred['pred'].copy()

In [ ]:
result.reset_index(drop=True, inplace=True)

In [ ]:
result.head()